In [1]:
import os
from typing import List

import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

DATA_PATH = r'CourseOCRTask3'
TARGET_PATH = r'data'

In [2]:
def get_mask(coordinates: np.ndarray, image_size: tuple) -> np.ndarray:
    mask = np.zeros(image_size, dtype=np.float32)
    points = coordinates.reshape((1, 4, 2)).astype(np.int32)
    cv2.fillPoly(mask, points, 1)
    return mask

def convert_data(image_dir: str, img_names: List[str], markup: pd.DataFrame, target_dir: str):
    target_image_dir = os.path.join(target_dir, 'images')
    target_mask_dir = os.path.join(target_dir, 'annotations')
    os.makedirs(target_image_dir, exist_ok=True)
    os.makedirs(target_mask_dir, exist_ok=True)

    for img_name in tqdm(img_names):
        if img_name not in markup.index:
            continue
        image = cv2.imread(os.path.join(image_dir, img_name), cv2.IMREAD_COLOR)
        mask = get_mask(markup.loc[img_name, ['x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4']].values, image.shape[:2])
        cv2.imwrite(os.path.join(target_image_dir, img_name), image)
        cv2.imwrite(os.path.join(target_mask_dir, img_name), mask)

def prepare_data(source_dir: str, target_dir: str) -> None:
    image_dir = os.path.join(source_dir, 'Images')
    img_names = os.listdir(image_dir)
    markup = pd.read_csv(os.path.join(source_dir, 'markup.csv'), header=None, index_col=0, encoding='utf-16',
                         names=['image_name', 'code', 'x1', 'y1',  'x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'markup'])

    convert_data(image_dir, img_names, markup, target_dir)

In [3]:
prepare_data(os.path.join(DATA_PATH, 'Train'), os.path.join(TARGET_PATH, 'Train'))

  0%|          | 0/8391 [00:00<?, ?it/s]

In [4]:
prepare_data(os.path.join(DATA_PATH, 'Test'), os.path.join(TARGET_PATH, 'Test'))

  0%|          | 0/100 [00:00<?, ?it/s]